<a href="https://colab.research.google.com/github/apurbaaaa/Fine-Tuning-Llama2/blob/main/Fine_tuning_Llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install -q accelerate peft bitsandbytes transformers trl sentence-transformers

In [19]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [20]:
 model_name = "NousResearch/Llama-2-7b-chat-hf"

 dataset_name = "mlabonne/guanaco-llama2-1k"

 new_model = "Llama-2-7b-chat-finetune"

In [21]:
#QLoRA params required

#attention dimension, Rank
lora_r = 64

#alpha parameter for scaling
lora_alpha = 16

#Drop
lora_dropout = 0.1

#for quantization, bitsnandbytes used
use_4bit = True
#Aactivate 4 bit precision base models
bnb_4bit_compute_dtype = "float16"

#Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

#Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [22]:
output_dir = "./results"

#number of training epochs
num_train_epochs = 1

#Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

#Batch size for training
per_device_train_batch_size = 4

#Batch size for eval
per_device_eval_batch_size = 4

#number of steps to accumulate gradients for
gradient_accumulation_steps = 1

#Enable gradient checkpointing
gradient_checkpointing = True

#Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

learning_rate = 2e-4

#Weight decay to apply to all laters except bias/LayerNorm weights
weight_decay = 0.001

#optimizer
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_schedule_type = "cosine"

#Number of training steps (overrides num_train_epochs)
max_steps = -1

#Ratio of steps for a linear warmup
warmup_ratio = 0.03

In [23]:
#Group sequences into batch with same lenght
#saves memory and speeds up training

group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X uupdates steps
logging_steps = 25

In [24]:
#SFT Params
max_seq_length = None

#pack multiple short exmples in the same input sequence to increase efficiency
packing = False

# Load model on GPU 0
devcice_map = {"": 0}

In [26]:
dataset = load_dataset(
    "mlabonne/guanaco-llama2-1k",
    split="train",
    streaming=True
)

#loading tokenizer and model with QLoRA config
compute_dtpe = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type = bnb_4bit_quant_type,
    bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant = use_nested_quant,
)

#Check GPU compatibilty with bfloat16

if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

#load llama tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#load lora config
peft_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    r = lora_r,
    bias = "none",
    task_type = "CAUSAL_LM",
)

#Set training parameters
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    optim = optim,
    save_steps = save_steps,
    logging_steps = logging_steps,
    learning_rate = learning_rate,
    weight_decay = weight_decay,
    fp16 = fp16,
    bf16 = bf16,
    max_grad_norm = max_grad_norm,
    max_steps = max_steps,
    warmup_ratio = warmup_ratio,
    group_by_length = group_by_length,
    lr_scheduler_type = lr_scheduler_type,
    report_to = "tensorboard"
)

#set supervised fine-tuning parameters
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    peft_config = peft_config,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = training_arguments,
    packing = packing,
)

#Train model
trainer.train()

NotImplementedError: Loading a streaming dataset cached in a LocalFileSystem is not supported yet.